## Setup

Requires a HubSpot Private app with an API access token with the following scopes:

- `crm.objects.custom.read`
- `crm.objects.companies.read`
- `crm.schemas.contacts.read`

The token should be stored in an environment variable called `HUBSPOT_ACCESS_TOKEN`.

You can copy the sample environment file to get started; run the following command from the root of this repository:

```bash
cp .env.sample .env
```

Then open `.env` and fill in with your access token.

In [ ]:
import os

from hubspot import HubSpot

from data.utils import hubspot_to_df, write_json_records


ACCESS_TOKEN = os.environ["HUBSPOT_ACCESS_TOKEN"]

hubspot = HubSpot(access_token=ACCESS_TOKEN)

In [ ]:
props_response = hubspot.crm.properties.core_api.get_all("notes", archived=False)
props_df = hubspot_to_df(props_response)
write_json_records(props_df, "notes_props.json")

In [ ]:
notes_props = ["hs_created_by", "hs_createdate", "hs_note_body"]
notes_assoc = ["companies", "contacts", "vendors"]

response = hubspot.crm.objects.notes.basic_api.get_page(limit=10, properties=notes_props, associations=notes_assoc)

notes = hubspot_to_df(response)
# drop notes without a body
notes = notes.dropna(subset=["properties.hs_note_body"])

# drop notes without a creator
notes = notes.dropna(subset=["properties.hs_created_by"])

# drop notes without any of the association types (e.g. all are NA)
notes = notes.dropna(subset=["associations.contacts.results", "associations.companies.results"], how="all")

In [ ]:
notes.info()